# Quick Start

In [ ]:
!pip install openai==1.57.0
!pip install langchain==0.3.10
!pip install langchain-community==0.3.10
!pip install langchain-core==0.3.22
!pip install langchain-openai==0.2.11

## 典型用法

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("对技术术语做名词解释：{name}?")
chat = ChatOpenAI(model_name="openai/gpt-4o-mini")
parser = StrOutputParser()

chain = prompt | chat | parser

res = chain.invoke({"name": "微服务架构"})
print(res)

微服务架构是一种软件架构风格，它将应用程序拆分成一系列小的、独立的服务，每个服务都围绕特定的业务功能构建。这些服务可以独立部署、扩展和更新，并通过轻量级的通信协议（如HTTP/REST、消息队列等）进行交互。

微服务架构的主要特点包括：

1. **模块化**：每个微服务都专注于某一特定的功能，具有清晰的边界和职责，便于理解和维护。

2. **独立性**：各个微服务可以独立开发、测试和部署，不会影响到其他服务。

3. **技术多样性**：不同的微服务可以使用不同的编程语言、框架和技术栈，从而选择最适合特定需求的工具。

4. **弹性和可扩展性**：微服务可以根据负载进行水平扩展，能够更好地应对流量变化。

5. **持续交付**：微服务架构支持持续集成和持续交付（CI/CD），使得新功能和修复能够快速发布。

6. **故障隔离**：由于服务之间的独立性，某个服务的故障不会直接影响到整个应用，这提高了系统的稳定性和可靠性。

微服务架构常用于构建复杂的分布式系统，尤其适合需要快速迭代和频繁发布的场景。


## 基本使用

In [5]:
# 将 OpenAI API 的调用经验迁移过来

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model_name = "openai/gpt-4o-mini"
llm = ChatOpenAI(model_name=model_name, temperature=0.8)

messages = [
    SystemMessage(content="You are an excellent translator, you are very at translating between English and Chinese"),
    # HumanMessage(content="Welcome to LLM development with langchain"),
    HumanMessage(content="欢迎使用 Langchain 进行大模型开发")
]

# 常规输出
res = llm.invoke(messages)

print(res.content)

Welcome to using Langchain for large model development!


In [7]:
# 流式输出
res = llm.stream(messages)

for chunk in res:
    print(chunk.content, end="|")


|Welcome| to| using| Lang|chain| for| large| model| development|!|||

In [10]:
# PromptTemplate

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个优秀的翻译专家，非常擅长把中文翻译为{language}"),
        ("user", "{text}")
    ]
)

prompt = prompt_template.invoke({"language": "英文", "text": "欢迎使用 Langchain 进行大模型开发"})

print(prompt, "\n")

# prompt.to_messages() 返回符合上一节预期的 message
print(prompt.to_messages(), "\n")

model_name = "openai/gpt-4o-mini"
chat = ChatOpenAI(model_name=model_name)

# 用于上一节相同的方式进行问答
res = chat.invoke(prompt.to_messages())
print(res.content, "\n")

# 简化写法
res = chat.invoke(prompt)
print(res.content, "\n")


messages=[SystemMessage(content='你是一个优秀的翻译专家，非常擅长把中文翻译为英文', additional_kwargs={}, response_metadata={}), HumanMessage(content='欢迎使用 Langchain 进行大模型开发', additional_kwargs={}, response_metadata={})] 

[SystemMessage(content='你是一个优秀的翻译专家，非常擅长把中文翻译为英文', additional_kwargs={}, response_metadata={}), HumanMessage(content='欢迎使用 Langchain 进行大模型开发', additional_kwargs={}, response_metadata={})] 

Welcome to using Langchain for large model development! 

Welcome to using Langchain for large model development! 



## Structured Output

### Using Json Mode

In [14]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

model_name = "openai/gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model_name,
    model_kwargs={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "email_schema",
                "schema": {
                    "type": "object",
                    "properties": {
                        "email": {
                            "description": "The email address that appears in the input",
                            "type": "string"
                        },
                        "additionalProperties": False
                    }
                }
            }
        }
    }
)

resp = chat.invoke([
    SystemMessage(content="You extract email addresses into JSON data."),
    HumanMessage(content="Feeling stuck? Send a message to help@mycompany.com.")
])

print(resp.content)

{"email":"help@mycompany.com"}


In [18]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")
chat.bind(response_format={
    "type": "json_schema",
    "json_schema": {
        "name": "email_schema",
        "schema": {
            "type": "object",
            "properties": {
                "email": {
                    "description": "The email address that appears in the input",
                    "type": "string"
                },
                "additionalProperties": False
            }
        }
    }
})

resp = chat.invoke([
    SystemMessage(content="You extract email addresses into JSON data."),
    HumanMessage(content="Feeling stuck? Send a message to help@mycompany.com.")
])

print(resp.content)

```json
{
  "email": "help@mycompany.com"
}
```


In [17]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")
chat.bind(response_format={"type": "json_object"})

resp = chat.invoke([
    SystemMessage(content="You extract email addresses into JSON data."),
    HumanMessage(content="Feeling stuck? Send a message to help@mycompany.com.")
])

print(resp.content)

```json
{
  "email": "help@mycompany.com"
}
```


In [19]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="openai/gpt-4o-mini",
                  model_kwargs={
                      "response_format": {"type": "json_object"}
                  })
chat.bind(response_format={"type": "json_object"})

resp = chat.invoke([
    SystemMessage(content="You extract email addresses into JSON data."),
    HumanMessage(content="Feeling stuck? Send a message to help@mycompany.com.")
])

print(resp.content)

{
  "email_addresses": [
    "help@mycompany.com"
  ]
}


### Using Structured Output

In [21]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel

class Email(BaseModel):
    email: str

chat = ChatOpenAI(model_name="openai/gpt-4o-mini").with_structured_output(Email)

resp = chat.invoke([
    SystemMessage(content="You extract email addresses into JSON data."),
    HumanMessage(content="Feeling stuck? Send a message to help@mycompany.com.")
])

print(resp, "\n\n", type(resp))

email='help@mycompany.com' 

 <class '__main__.Email'>


In [20]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel


class Step(BaseModel):
    explanation: str
    output: str


class MathReasoning(BaseModel):
    steps: list[Step]
    final_output: str


chat = ChatOpenAI(model_name="openai/gpt-4o-mini").with_structured_output(MathReasoning)
resp = chat.invoke([
    {"role": "system",
     "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
    {"role": "user", "content": "how can I solve 8x + 7 = -25"}
])

print(resp, "\n", type(resp))


steps=[Step(explanation='Subtract 7 from both sides of the equation to isolate the term with x.', output='8x + 7 - 7 = -25 - 7'), Step(explanation='This simplifies to 8x = -32.', output='8x = -32'), Step(explanation='Now, divide both sides by 8 to solve for x.', output='x = -32 / 8'), Step(explanation='This simplifies to x = -4.', output='x = -4')] final_output='The solution to the equation 8x + 7 = -25 is x = -4.' 
 <class '__main__.MathReasoning'>


### Using Tool calling

In [33]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class Email(BaseModel):
    """Use this tool to get email structure"""
    email: str = Field(description="input email address here")

chat = ChatOpenAI(model_name="openai/gpt-4o-mini", temperature=0).bind_tools([Email])

resp = chat.invoke([
    SystemMessage(content="You extract email addresses into JSON data."),
    HumanMessage(content="Feeling stuck? Send a message to help@mycompany.com.")
])

print(resp, "\n\n", type(resp))

content='' additional_kwargs={'tool_calls': [{'id': 'call_ICigxmfrtzPCXUIjW9awzIBD', 'function': {'arguments': '{"email":"help@mycompany.com"}', 'name': 'Email'}, 'type': 'function', 'index': 0}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 71, 'total_tokens': 88, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'openai/gpt-4o-mini', 'system_fingerprint': 'fp_818c284075', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-737bb30a-56c4-48ef-9ee5-6fb0c279a893-0' tool_calls=[{'name': 'Email', 'args': {'email': 'help@mycompany.com'}, 'id': 'call_ICigxmfrtzPCXUIjW9awzIBD', 'type': 'tool_call'}] usage_metadata={'input_tokens': 71, 'output_tokens': 17, 'total_tokens': 88, 'input_token_details': {}, 'output_token_details': {}} 

 <class 'langchain_core.messages.ai.AIMessage'>


In [44]:
print(resp.additional_kwargs["tool_calls"][0])
args = resp.additional_kwargs["tool_calls"][0]["function"]["arguments"]
print(args)

{'id': 'call_Z082gPUmHrMaToAC4uAuyVvp', 'function': {'arguments': '{"steps":[{"explanation":"Subtract 7 from both sides: 8x + 7 - 7 = -25 - 7","output":"8x = -32"},{"explanation":"Divide both sides by 8: x = -32 / 8","output":"x = -4"}],"final_output":"x = -4"}', 'name': 'MathReasons'}, 'type': 'function', 'index': 0}
{"steps":[{"explanation":"Subtract 7 from both sides: 8x + 7 - 7 = -25 - 7","output":"8x = -32"},{"explanation":"Divide both sides by 8: x = -32 / 8","output":"x = -4"}],"final_output":"x = -4"}


In [45]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class MathStep(BaseModel):
    explanation: str
    output: str


class MathReasons(BaseModel):
    """Use this tool to explain math steps and get final result"""
    steps: list[MathStep] = Field(description="record every math step here")
    final_output: str = Field(description="record the final result")


chat = ChatOpenAI(model_name="openai/gpt-4o-mini").bind_tools([MathReasons])
resp = chat.invoke([
    {"role": "system",
     "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
    {"role": "user", "content": "how can I solve 8x + 7 = -25"}
])

print(resp, "\n\n", type(resp))

content="To solve the equation \\(8x + 7 = -25\\), we can follow these steps:\n\n1. **Subtract 7 from both sides**: This will help isolate the term with \\(x\\) on one side of the equation.\n   \n   \\[\n   8x + 7 - 7 = -25 - 7\n   \\]\n\n2. **Simplify both sides**: After performing the subtraction, simplify the equation.\n\n3. **Divide by 8**: Once \\(x\\) is isolated, divide both sides by 8 to solve for \\(x\\).\n\nLet's go through these steps one by one. \n\n1. Subtract 7 from both sides:\n\n   \\[\n   8x = -25 - 7\n   \\]\n\n2. Simplifying the right side:\n\n   \\[\n   8x = -32\n   \\]\n\n3. Now, divide both sides by 8:\n\n   \\[\n   x = \\frac{-32}{8}\n   \\]\n\n4. Simplifying the fraction:\n\n   \\[\n   x = -4\n   \\]\n\nThus, the solution to the equation \\(8x + 7 = -25\\) is \\(x = -4\\).\n\nNow, let's record these steps and the final result." additional_kwargs={'tool_calls': [{'id': 'call_BEIqQsiQx5e23GqBn0CbxTqr', 'function': {'arguments': '{"steps":[{"explanation":"Subtract 

In [55]:
args = resp.additional_kwargs["tool_calls"][0]["function"]["arguments"]
print(args)

# MathStep.model_validate(obj)

{"steps":[{"explanation":"Subtract 7 from both sides of the equation: 8x + 7 - 7 = -25 - 7","output":"8x = -32"},{"explanation":"Simplify the right side of the equation: 8x = -32","output":"8x = -32"},{"explanation":"Divide both sides by 8 to isolate x: x = -32 / 8","output":"x = -4"}],"final_output":"x = -4"}


# ModelIO

- Format: PromptTemplate
- Predict: ChatModel/LLM
- Parse: OutputParser

In [ ]:
## Quick Start

## PromptTemplate

## ChatModel

## OutputParser